In [ ]:
%matplotlib inline

import abtem
import ase
import matplotlib.pyplot as plt
import numpy as np
from ase.visualize import view
from ase.build import surface

### Read cif file

In [ ]:
PbTe = ase.io.read("PbTe.cif")
abtem.show_atoms(PbTe);

### Create a 110 surface

In [ ]:
PbTe_110 = surface(PbTe, indices=(1, 1, 0), layers=3, periodic=True)

abtem.show_atoms(PbTe_110, plane="xy");

### Expand the cell

In [ ]:
repeated_PbTe = PbTe_110.copy()

repeated_PbTe *= (4, 6, 4)

abtem.show_atoms(repeated_PbTe, legend=True);

In [ ]:
abtem.show_atoms(repeated_PbTe, legend=True, plane='xz');

In [ ]:
from ase.io import write

write("PbTe_110.cif", repeated_PbTe)

### Build Atomic Potential

In [ ]:
%matplotlib ipympl

import abtem
import ase
import matplotlib.pyplot as plt
import numpy as np

# np.set_printoptions(edgeitems=2)

abtem.config.set({"visualize.cmap": "plasma"})
abtem.config.set({"visualize.continuous_update": True})
abtem.config.set({"visualize.autoscale": True})
# abtem.config.set({"visualize.reciprocal_space_units": "mrad"})
abtem.config.set({"device": "cpu"})
abtem.config.set({"fft": "fftw"});

In [ ]:
PbTe_110 = ase.io.read("PbTe_110.cif")

fig, (ax1, ax2) = plt.subplots(1, 2)
abtem.show_atoms(PbTe_110, ax=ax1, plane="xy", title="Beam view", legend=True)
abtem.show_atoms(PbTe_110, ax=ax2, plane="yz", title="Side view");

### Set the sampling rate

In [ ]:
potential = abtem.Potential(PbTe_110, sampling=0.05, slice_thickness=2)

In [ ]:
len(potential)

### Compute the potential

In [ ]:
potential_array = potential.build()

In [ ]:
potential_array.array

In [ ]:
potential_array.compute()

In [ ]:
potential_array.show();

In [ ]:
potential_array.to_images().show(interact=True, cbar=True);

In [ ]:
probe = abtem.Probe(energy=200e3, defocus=25, semiangle_cutoff=27)

In [ ]:
probe.grid.match(potential)

In [ ]:
probe_waves = probe.build()

probe_waves.array

In [ ]:
probe_waves.compute()

In [ ]:
probe_waves.intensity().show(title="Probe intensity")

In [ ]:
probe_waves.diffraction_patterns().show(
    title="Reciprocal-space probe intensity", units="mrad"
);

### Set the scanning grid

In [ ]:
scan = abtem.GridScan(
    start=(0, 0),
    end=(1/8, 1/6),
    sampling=probe.ctf.nyquist_sampling,
    fractional=True,
    potential=potential,
)

In [ ]:
fig, ax = abtem.show_atoms(PbTe_110)
scan.add_to_plot(ax, color="k");

### Set the detector

In [ ]:
bright = abtem.AnnularDetector(inner=0, outer=20)
maadf = abtem.AnnularDetector(inner=50, outer=120)
haadf = abtem.AnnularDetector(inner=100, outer=180)

detectors = [bright, maadf, haadf]

In [ ]:
print(f"alpha_max = {min(probe.cutoff_angles):.1f} mrad")

In [ ]:
bright_region = bright.get_detector_region(probe)
maadf_region = maadf.get_detector_region(probe)
haadf_region = haadf.get_detector_region(probe)

In [ ]:
stacked_regions = abtem.stack(
    (bright_region, maadf_region, haadf_region), ("bright", "MAADF", "HAADF")
)

visualization = stacked_regions.show(explode=True, units="mrad", figsize=(12, 4))

In [ ]:
scanned_measurements = probe.scan(
    scan=scan,
    detectors=detectors,
    potential=potential,
)

scanned_measurements.compute()

In [ ]:
stacked_measurements = abtem.stack(scanned_measurements, ("BF", "MAADF", "HAADF"))

stacked_measurements.show(explode=True);

### Interpolate the result

In [ ]:
interpolated_measurements = stacked_measurements.interpolate(0.1)

interpolated_measurements.show(explode=True);

### Blurring

In [ ]:
blurred_measurements = interpolated_measurements.gaussian_filter(0.35)

blurred_measurements.show(explode=True);

In [ ]:
tiled_measurements = blurred_measurements.tile((20, 14))

In [ ]:
noisy_measurements = tiled_measurements.poisson_noise(dose_per_area=5e5, seed=100)

noisy_measurements.show(explode=True, figsize=(12, 4),cmap='gray');

In [ ]:
noisy_measurements[2].show(figsize=(10,10),cmap='gray')